In [44]:
import glob
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize

stemmer = LancasterStemmer()

In [45]:
def clean_html(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [46]:
def stem_text(text):
    words = word_tokenize(text)
    result = ""
    for word in words:
        result += stemmer.stem(word) + " "
    return result

In [47]:
with open("spam-mail.tr.label", "r") as file:
    lines = file.readlines()
    
lines = [l.strip() for l in lines[1:]]
labels = {}
for line in lines:
    x = int(line.split(',')[0])
    y = int(line.split(',')[1])
    labels[x] = y

In [48]:
email_folder = "tr"
emails = glob.glob(email_folder + "/*")

train_x = []
train_y = []
test_x = []
test_y = []

for email in emails[:int(0.8* len(emails))]:
    with open(email, "r") as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
        email_stemmed = stem_text(email_clean)
    train_x.append(email_stemmed)
    id = int(email.split('.')[0].split('_')[1])
    train_y.append(labels[id])

for email in emails[int(0.8* len(emails)):]:
    with open(email, "r") as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
        email_stemmed = stem_text(email_clean)
    test_x.append(email_stemmed)
    id = int(email.split('.')[0].split('_')[1])
    test_y.append(labels[id])

In [57]:
count_vect = CountVectorizer(stop_words= 'english',min_df=10, ngram_range= (1, 2))
# tfidf_transformer = TfidfTransformer(use_idf= False)
train_x_count = count_vect.fit_transform(train_x)
# train_x_tf_idf = tfidf_transformer.fit_transform(train_x_count)

In [58]:
clf = MultinomialNB().fit(train_x_count, train_y)

In [59]:
test_x_count = count_vect.transform(test_x)
# test_x_tf_idf = tfidf_transformer.transform(test_x_count)
predicted = clf.predict(test_x_count)

count_true = 0
count_false = 0
for pred, label in zip(predicted, test_y):
    if pred == label:
        count_true += 1
    else:
        count_false += 1

In [60]:
print(classification_report(y_true= test_y, y_pred= predicted, digits= 4))

             precision    recall  f1-score   support

          0     0.9682    0.9500    0.9590       160
          1     0.9766    0.9853    0.9809       339

avg / total     0.9739    0.9739    0.9739       499

